In [689]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os
from os import path
import glob
import xlsxwriter
import openpyxl # 3.0
import re
import datetime as dt

## Import des données 

In [690]:
%%time
temps = pd.read_excel("Data/Dim temps.xlsx")
offres = pd.read_excel("Data/PBI Offres.xlsx")
candidature = pd.read_excel("Data/PBI Candidatures.xlsx")
candidat = pd.read_excel('Data/PBI Candidats.xlsx')

Wall time: 2min 43s


## Déclaration des fonctions personnalisées utilisées

In [691]:
# La fonction indicator permet de rattacher un évènement temporel (changement de statut, date de publication) à la dernière date du mois durant laquelle l'évènement s'est produit.
# Ceci permet de réaliser des jointures dans la table de fait pour rattacher l'évènement à une ligne.

def indicator_dt(dataframe_source, champ_source, nom_champ_cible):
    dataframe_source[nom_champ_cible] = dataframe_source[champ_source].apply(lambda x: x.replace(day=x.days_in_month))
    dataframe_source[nom_champ_cible] = dataframe_source[nom_champ_cible].astype(str)
    dataframe_source[nom_champ_cible] = dataframe_source[nom_champ_cible].apply(lambda x: x[:10])

# la fonction link_count est utilisée pour calculer les indicateurs. Elle permet de rattacher les date d'une source de données à des dates fin de mois puis de les compter (groupby).
    
def link_count( alim_dataFrame, target_dataFrame, left_keys, event_date, indicator_name, right_keys=None, type_conso=None):
    # alim_dataFrame dataframe à alimenter
    # target_dataFrame: dataframe contenant l'indicateur à récupérer
    # left_keys : clés servant à réaliser la jointure sur le dataframe à alimenter
    # event_date : date de l'
    # event_name: nom de l'indicateur cible 
    # right_key : facultatif, nom des clés de jointure de droite dans le cas où ces dernière diffère de celle de gauche
    
    tr_target = target_dataFrame.copy()
    tr_target["link_date"] = tr_target[event_date].apply(lambda x: x.replace(day=x.days_in_month))
    tr_target["link_date"] = tr_target["link_date"].astype(str)
    tr_target["link_date"] = tr_target["link_date"].apply(lambda x: x[:10])
    
    tr_alim = alim_dataFrame[left_keys]

    if right_keys == None:
        right_keys = left_keys.copy()[:len(left_keys)-1]
        right_keys.append("link_date")    
        result = tr_alim.merge(tr_target[right_keys], right_on = right_keys, left_on = left_keys, how='left')
        
        if type_conso == None:
            result = result.groupby(left_keys)["link_date"].count().reset_index()
        if type_conso == "unique":
            result = result.groupby(left_keys)["link_date"].nunique().reset_index()
            
            print(result)
            
        result[indicator_name] = result["link_date"]
        result.drop(["link_date"], axis=1, inplace=True)
        alim_dataFrame = alim_dataFrame.merge(result, on=left_keys, how='left')     
        
    else:
        pass
            
    return alim_dataFrame

## Transformation des données sources

### Préparation de la table offres

In [692]:
%%time
# Remarques: Les données offres comportent des doublons. Une offre est publiée par exemple plusieurs fois le même jour, parfois même à la même minute.
# Des règles sont appliquées afin de rendre unique la combinaison IdDemandeAuto, Date de publication de l'offre, Date de clôture réelle

# Ajout d'une date fictive pour les offres ayant une date de publication de l'offre vide
offres_prep = offres.copy()
offres_prep.loc[offres_prep["Date de publication de l'offre"].isna()==True,"Date de publication de l'offre"] =dt.date(1900, 1, 1)

# Ajout du champ date de candidature contenant uniquement la partie date du champ Date de publication de l'offre
offres_prep["date de publication"] = offres_prep["Date de publication de l'offre"].dt.date

#Suppression des lignes doublons
offres_prep = offres_prep.drop_duplicates()

# Suppression des Date de publication identiques (on conserve pour un même jour que l'heure la plus récente ): regroupement des données par "IdDemandeAuto","date candidature" et max "Date de publication de l'offre" afin ne conserver que les candidatures les plus récentes chauque jour
to_keep = offres_prep.groupby(["IdDemandeAuto","date de publication"])["Date de publication de l'offre"].max().reset_index()
to_keep = to_keep.merge(offres_prep[["IdDemandeAuto", "date de publication","Date de publication de l'offre","Date de clôture réelle"]], on=["IdDemandeAuto","date de publication","Date de publication de l'offre"], how="left")

# Lorsque date et heure de candidature identiques, on conserve la ligne avec la date de clôture réelle la plus grande
to_keep = to_keep.groupby(["IdDemandeAuto","date de publication","Date de publication de l'offre"])["Date de clôture réelle"].max().reset_index()

# Jointure avec la table offres afin de récupérer les autres données
offres_prep = to_keep.merge(offres_prep, on=["IdDemandeAuto","date de publication","Date de publication de l'offre","Date de clôture réelle"], how="left")
offres_prep.reset_index(drop=True, inplace=True)

#Remplacement des valeurs Direction Gaz et Energie production par "Gas Renewables And Power" dans branche
offres_prep["Branche"]= offres_prep["Branche"].apply(lambda x: "Gas Renewables And Power" if x in ["Direction Gaz", "Energies Nouvelles"] else x)

# clé permettant un outer join avec la table fact_temps
offres_prep["custom"] = 1

# Ajout d'une clé technique offre
offres_prep["Key_offre"] = offres_prep.index

Wall time: 532 ms


### Préparation de la table candidatures

In [693]:
%%time
# Filtre sur les candidatures internes

candidature_prep = candidature.loc[candidature["Type"]=="Interne"].copy()
source1 = candidature_prep

# Ajout de la clé offre
candidature_prep = candidature_prep.merge(offres_prep[["IdDemandeAuto", "Key_offre","Date de publication de l'offre",  "Date de clôture réelle"]], on=["IdDemandeAuto"], how="left")
candidature_prep = candidature_prep.loc[(candidature_prep["Date du dépôt de candidature"]>=candidature_prep["Date de publication de l'offre"]) & (candidature_prep["Date du dépôt de candidature"]<=candidature_prep["Date de clôture réelle"])]

#df = df1.merge(df2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']


Wall time: 316 ms


### Préparation de la table candidat

In [694]:
%%time
candidat_prep = candidat.copy()
candidat_prep = candidat_prep[['IdCandidat','Nom', 'Prénom','Type actuel candidat']]
candidat_prep = candidat_prep.drop_duplicates()

# Seul les candidats intenes sont retenus
candidat_prep = candidat_prep.loc[candidat_prep["Type actuel candidat"] == "Interne"]

# Les candidats sont identifiés par la concaténation du nom et du prénom comme vu en atelier
candidat_prep = candidat_prep[["IdCandidat", "Nom", "Prénom"]]
candidat_prep["Concat_name"] = candidat_prep["Nom"].astype('str') + " " + candidat_prep["Prénom"].astype('str')
index_name = pd.DataFrame({"Concat_name" : candidat_prep["Concat_name"].unique()})
index_name["Id_Cand"] = index_name.index

candidat_prep = candidat_prep.merge(index_name, on="Concat_name", how='left')
candidat_prep=candidat_prep.rename(columns={"IdCandidat" :"Num. de référence du candidat"})

Wall time: 256 ms


## Création des dimensions

### Création de la dimension offre

In [695]:
%%time
dim_offres = offres_prep.copy()

# Ajout de la durée de publication
depubliees_nonvide = dim_offres.loc[dim_offres["Date de clôture réelle"].isna()==False].copy()
depubliees_nonvide["Durée publication"] = (depubliees_nonvide["Date de clôture réelle"] - depubliees_nonvide["Date de publication de l'offre"]).dt.days
dim_offres = dim_offres.merge(depubliees_nonvide[["Key_offre", "Durée publication"]], on=["Key_offre"], how='left')

# Ajout de la durée de staffing
affect_nonvide = candidature_prep.copy()
affect_nonvide = affect_nonvide.loc[affect_nonvide["Interne - Affecté au poste"].isna()==False]

affect_nonvide=affect_nonvide.groupby(["Key_offre"])["Interne - Affecté au poste"].min().reset_index()
affect_nonvide = affect_nonvide.merge(dim_offres[["Key_offre", "Date de publication de l'offre"]], on=["Key_offre"], how='left')
affect_nonvide["Durée staffing"] = (affect_nonvide["Interne - Affecté au poste"] - affect_nonvide["Date de publication de l'offre"]).dt.days

dim_offres = dim_offres.merge(affect_nonvide[["Key_offre", "Durée staffing", "Interne - Affecté au poste"]], on=["Key_offre"], how='left')

Wall time: 397 ms


### Création de la dimension temps et du DataFrame fact_temps utilisé comme grain temps dans les tables de faits

In [696]:
%%time
# Création de la table de temps. Nous gardons que les 2 dernières années
dim_temps =  temps[["date key","full date", "day num in month", "month", "month name", "year","month and year"]].copy()

# Détermination de l'année actuelle
now = dt.datetime.now()
actual_year = now.year
actual_month = now.month
last_day=pd.Period(str(now)).days_in_month
now = dt.date(actual_year,actual_month, last_day)
now = np.datetime64(now)

# dim_temps est filtrée sur l'année courante et l'année précédente
dim_temps = dim_temps.loc[(dim_temps["year"]==actual_year) | (dim_temps["year"] == actual_year-1)]

# Les dates portants sur le futur ne sont pas retenues sur l'année en cours
dim_temps = dim_temps.loc[dim_temps["full date"] <= now ]

# Création de la table fact_temps permettant de créer la table de fait
fact_temps = dim_temps[["year", "month","day num in month"]]
fact_temps = fact_temps.groupby(["year", "month"])["day num in month"].max().reset_index()

fact_temps = fact_temps.merge(dim_temps[["year", "month", "day num in month", "full date", "date key","month and year"]], how='left', on=["year", "month","day num in month"])
fact_temps["custom"] = 1


Wall time: 28.2 ms


### Création de la dimension candidat

In [697]:
dim_cand = candidat_prep.copy()

## Création des tables faits

### Création de la table de faits fact_offres

In [698]:
%%time
# Création de la table de fait offres d'emploi: Grain = 1 offre (IdDemandeAuto, date de ) par mois
fact_offres = dim_offres[["Key_offre", "IdDemandeAuto","date de publication","custom"]].copy()
fact_offres = fact_offres.drop_duplicates()
fact_offres = fact_offres.merge(dim_offres[["Key_offre","Date de publication de l'offre", "Date de clôture réelle", "Date d'ouverture", "Date de fermeture"]], on=["Key_offre"] ,how='left')
fact_offres = fact_offres.merge(fact_temps[["date key","full date","custom"]], on=["custom"], how = 'outer')
fact_offres["full date link"] = fact_offres["full date"].astype(str) # Full date en string pour jointure

Wall time: 4.44 s


In [699]:
%%time
# indicateur nombre de publication
fact_offres = link_count(fact_offres, dim_offres,["Key_offre","full date link"], "Date de publication de l'offre", "Nombre de publications")
# indicateur nombre de dépublication
fact_offres = link_count(fact_offres, dim_offres,["Key_offre","full date link"], "Date de clôture réelle", "Nombre de dépublications")
# indicateur nombre d'offre étant publiées
date_exclue= str(dt.date(1900, 1, 1))
fact_offres.loc[(fact_offres["Date de publication de l'offre"]<= fact_offres["full date"]) & ((fact_offres["Date de clôture réelle"]>fact_offres["full date"]) | ((fact_offres["Date de clôture réelle"].isna()==True) & (fact_offres["date de publication"].astype('str') != date_exclue))),"Nombre d'offres étant publiées"]=1
# indicateur nombre d'offre étant dépubliées
fact_offres.loc[fact_offres["Date de clôture réelle"]<=fact_offres["full date"],"Nombre d'offres étant dépubliées"]=1
# Termoin publié
fact_offres.loc[fact_offres["Nombre d'offres étant publiées"]==1, "Témoin publication"]= 1
fact_offres.loc[fact_offres["Témoin publication"] != 1, "Témoin publication"]= 0

# indicateurs d'ouverture/fermeture
#indcateur nombre d'offres ouvertes
fact_offres = link_count(fact_offres, dim_offres,["Key_offre","full date link"], "Date d'ouverture", "Nombre d'offres ouvertes")
#indicateur nombre d'offre fermées
fact_offres = link_count(fact_offres, dim_offres,["Key_offre","full date link"], "Date de fermeture", "Nombre d'offres fermées")
# indicateur nombre d'offres étant ouvertes
fact_offres.loc[(fact_offres["Date d'ouverture"]<= fact_offres["full date"]) & ((fact_offres["Date de fermeture"]>fact_offres["full date"]) | (fact_offres["Date de fermeture"].isna()==True)),"Nombre d'offres étant ouvertes"]=1
# indicateur nombre d'offres étant fermées
fact_offres.loc[fact_offres["Date de fermeture"]<=fact_offres["full date"],"Nombre d'offres étant fermées"]=1

Wall time: 8.56 s


In [700]:
%%time
#Nombre de candidatures
fact_offres = link_count(fact_offres, candidature_prep,["Key_offre","full date link"], "Date du dépôt de candidature", "Nombre de candidature")
#Nombre de candidatures cumulées
fact_offres["Nombre de candidatures cumulées"] = fact_offres.groupby(['Key_offre'])["Nombre de candidature"].cumsum()
#Nombre de candidats affectés au poste
fact_offres = link_count(fact_offres, candidature_prep,["Key_offre","full date link"], "Interne - Affecté au poste", "Nombre de candidats affectés au poste")
#Nombre de candidats en short list
fact_offres = link_count(fact_offres, candidature_prep,["Key_offre","full date link"], "Interne - Shortlist finale", "Nombre de candidats en short list")
# Nombre d'offres staffée
first_staff = fact_offres[(fact_offres["Nombre de candidats affectés au poste"]>0) & (fact_offres["Nombre d'offres étant fermées"]==1)].copy()
first_staff = first_staff.groupby(["Key_offre"])["full date"].min().reset_index()
first_staff["Nombre d'offres staffées"] = 1
fact_offres = fact_offres.merge(first_staff, on=["Key_offre","full date"], how='left')
#Nombre d'offres étant staffée 
fact_offres["Nombre d'offres étant staffées"] = fact_offres.groupby(['Key_offre'])["Nombre d'offres staffées"].cumsum()

Wall time: 5.37 s


In [701]:
fact_offres.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693405 entries, 0 to 693404
Data columns (total 26 columns):
Key_offre                                693405 non-null int64
IdDemandeAuto                            693405 non-null object
date de publication                      693405 non-null object
custom                                   693405 non-null int64
Date de publication de l'offre           693405 non-null datetime64[ns]
Date de clôture réelle                   640260 non-null datetime64[ns]
Date d'ouverture                         663675 non-null datetime64[ns]
Date de fermeture                        393525 non-null datetime64[ns]
date key                                 693405 non-null int64
full date                                693405 non-null datetime64[ns]
full date link                           693405 non-null object
Nombre de publications                   693405 non-null int64
Nombre de dépublications                 693405 non-null int64
Nombre d'offres étant 

In [702]:
%%time
# Typage des valeurs
fact_offres.loc[fact_offres["Nombre d'offres étant publiées"].isna()==True,"Nombre d'offres étant publiées"]=0
fact_offres["Nombre d'offres étant publiées"] = fact_offres["Nombre d'offres étant publiées"].astype('int64')

fact_offres.loc[fact_offres["Nombre d'offres étant dépubliées"].isna()==True,"Nombre d'offres étant dépubliées"]=0
fact_offres["Nombre d'offres étant dépubliées"] = fact_offres["Nombre d'offres étant dépubliées"].astype('int64')

fact_offres.loc[fact_offres["Nombre d'offres étant ouvertes"].isna()==True, "Nombre d'offres étant ouvertes"]=0
fact_offres["Nombre d'offres étant ouvertes"] = fact_offres["Nombre d'offres étant ouvertes"].astype('int64')

fact_offres.loc[fact_offres["Nombre d'offres étant dépubliées"].isna()==True, "Nombre d'offres étant dépubliées"]=0
fact_offres["Nombre d'offres étant dépubliées"] = fact_offres["Nombre d'offres étant dépubliées"].astype('int64') 

fact_offres.loc[fact_offres["Nombre d'offres étant fermées"].isna()==True, "Nombre d'offres étant fermées"]=0
fact_offres["Nombre d'offres étant fermées"] = fact_offres["Nombre d'offres étant fermées"].astype('int64') 

fact_offres.loc[fact_offres["Nombre d'offres staffées"].isna()==True, "Nombre d'offres staffées"]=0
fact_offres["Nombre d'offres staffées"] = fact_offres["Nombre d'offres staffées"].astype('int64')

fact_offres.loc[fact_offres["Nombre d'offres étant staffées"].isna()==True, "Nombre d'offres étant staffées"]=0
fact_offres["Nombre d'offres étant staffées"] = fact_offres["Nombre d'offres étant staffées"].astype('int64')

fact_offres["Témoin publication"] = fact_offres["Témoin publication"].astype('int64')

print(fact_offres.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693405 entries, 0 to 693404
Data columns (total 26 columns):
Key_offre                                693405 non-null int64
IdDemandeAuto                            693405 non-null object
date de publication                      693405 non-null object
custom                                   693405 non-null int64
Date de publication de l'offre           693405 non-null datetime64[ns]
Date de clôture réelle                   640260 non-null datetime64[ns]
Date d'ouverture                         663675 non-null datetime64[ns]
Date de fermeture                        393525 non-null datetime64[ns]
date key                                 693405 non-null int64
full date                                693405 non-null datetime64[ns]
full date link                           693405 non-null object
Nombre de publications                   693405 non-null int64
Nombre de dépublications                 693405 non-null int64
Nombre d'offres étant 

### Création de table de fait candidats

In [703]:

fact_candidat = candidature_prep[['Num. de référence du candidat','Key_offre','Date du dépôt de candidature']].copy()
fact_candidat = fact_candidat.merge(candidat_prep, on='Num. de référence du candidat', how='left')
fact_candidat["Nombre de candidature"] = 1
fact_candidat['Date du dépôt de candidature'] = fact_candidat['Date du dépôt de candidature'].astype('str')
fact_candidat['Date du dépôt de candidature'] = fact_candidat['Date du dépôt de candidature'].apply(lambda x: x[:10])

# Ajout de la clé temps
temps_cand = dim_temps.copy()
temps_cand["full date"] =temps_cand["full date"].astype('str')
fact_candidat = fact_candidat.merge(temps_cand[["date key","full date"]], left_on=['Date du dépôt de candidature'], right_on=['full date'], how='left')
# Suppression des candidatures n'étant pas ratachées à la période étudiée
fact_candidat = fact_candidat[fact_candidat["date key"].isna()== False]
# Suppression des champs inutiles
fact_candidat = fact_candidat[["Key_offre", "Num. de référence du candidat","Nombre de candidature"]]


In [ ]:
%%time
dim_offres.to_excel(r'Transformed data/dim_offres.xlsx')
fact_offres.to_excel(r'Transformed data/fact_offres.xlsx')
fact_candidat.to_excel(r'Transformed data/fact_candidat.xlsx')
dim_temps.to_csv('rTransformed data/dim_temps.csv', encoding='utf-8', index=False)
dim_offres.to_csv(r'Transformed data/dim_offres.csv', encoding='utf-8', index=False)
fact_offres.to_csv(r'Transformed data/fact_offres.csv', encoding='utf-8', index=False)
fact_candidat.to_csv(r'Transformed data/fact_candidat.csv', encoding='utf-8', index=False)

In [ ]:
#candidature_prep.to_excel(r'Transformed data/candidature_prep.xlsx')